In [2]:
!pip install backtrader

  Using cached backtrader-1.9.76.123-py2.py3-none-any.whl (410 kB)


In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import backtrader as bt
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

In [117]:
class TestStrategy(bt.Strategy):
    params = (
        ('exitbars', 5),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + self.params.exitbars):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()


In [121]:
cerebro = bt.Cerebro()

In [122]:
# Add a strategy
cerebro.addstrategy(TestStrategy)

0

In [123]:
data = bt.feeds.YahooFinanceCSVData(
        dataname="./data/BTC-USD.csv",
        # Do not pass values before this date
        fromdate=datetime.datetime(2021, 10, 10),
        # Do not pass values after this date
        todate=datetime.datetime(2021, 10, 10),
        reverse=False)
data

In [107]:
cerebro.adddata(data)

In [109]:
cerebro.broker.setcash(100000.0)

In [110]:
# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

In [111]:
cerebro.broker.setcommission(commission=0.001)

In [112]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00


In [113]:
cerebro.run()

In [114]:
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Final Portfolio Value: 100000.00


#### Include plots

In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import backtrader as bt
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

In [2]:
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [3]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

0

In [4]:
data = bt.feeds.YahooFinanceCSVData(
        dataname="./data/BTC-USD.csv",
        # Do not pass values before this date
        fromdate=datetime.datetime(2021, 10, 10),
        # Do not pass values after this date
        todate=datetime.datetime(2022, 10, 10),
        reverse=False)
data

In [5]:
# Add the Data Feed to Cerebro
cerebro.adddata(data)

In [6]:
# Set our desired cash start
cerebro.broker.setcash(1000.0)

In [7]:
# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

In [8]:
# Set the commission
cerebro.broker.setcommission(commission=0.0)

In [9]:
# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 1000.00


In [10]:
# Run over everything
cerebro.run()

2021-11-12, Close, 64155.94
2021-11-12, BUY CREATE, 64155.94
2021-11-13, Order Canceled/Margin/Rejected
2021-11-13, Close, 64469.53
2021-11-13, BUY CREATE, 64469.53
2021-11-14, Order Canceled/Margin/Rejected
2021-11-14, Close, 65466.84
2021-11-14, BUY CREATE, 65466.84
2021-11-15, Order Canceled/Margin/Rejected
2021-11-15, Close, 63557.87
2021-11-16, Close, 60161.25
2021-11-17, Close, 60368.01
2021-11-18, Close, 56942.14
2021-11-19, Close, 58119.58
2021-11-20, Close, 59697.20
2021-11-21, Close, 58730.48
2021-11-22, Close, 56289.29
2021-11-23, Close, 57569.07
2021-11-24, Close, 56280.43
2021-11-25, Close, 57274.68
2021-11-26, Close, 53569.77
2021-11-27, Close, 54815.08
2021-11-28, Close, 57248.46
2021-11-29, Close, 57806.57
2021-11-30, Close, 57005.43
2021-12-01, Close, 57229.83
2021-12-02, Close, 56477.82
2021-12-03, Close, 53598.25
2021-12-04, Close, 49200.70
2021-12-05, Close, 49368.85
2021-12-06, Close, 50582.62
2021-12-07, Close, 50700.09
2021-12-08, Close, 50504.80
2021-12-09, Clos

In [11]:
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Final Portfolio Value: 1000.00


In [14]:
# Plot the result
cerebro.plot()

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/opt/miniconda/lib/python3.9/site-packages/matplotlib/dates.py)

In [2]:
from notebook import notebookapp
servers = list(notebookapp.list_running_servers())
print(servers)


[]
